In [2]:
from PIL import Image, ImageOps
from numpy import asarray
import matplotlib.pyplot as plt
import numpy as np
from itertools import product
from PIL import Image
from numpy import asarray
import math
import os
from persim import plot_diagrams
from ripser import ripser, lower_star_img
import scipy
from scipy import ndimage

def red_regions_index(img_array, n):
    a = img_array.shape[0]
    b = img_array.shape[1]
    A = np.arange(a)
    B = np.arange(b)
    C = list(product(A, B))
    
    max_val = img_array[:,:,0].max()
    prob = img_array.astype(np.float64)
    prob = 1/max_val * img_array
    prob = np.exp((np.exp(np.exp(prob))))
    max_sum = np.sum(prob[:,:,0])
    prob = 1/max_sum * prob
    prob = prob[:,:,0].flatten()

    index_regions = np.random.choice(len(C), n, replace = False, p = prob)
    return index_regions, C

def red_regions(img_array, n):
    list_red_regions = []
    region_indexes, C = red_regions_index(img_array, n)
    for x in region_indexes:
        if len(img_array[C[x][0]:(C[x][0] + 10), C[x][1]:(C[x][1] + 10)]) == 10:
            list_red_regions.append(img_array[C[x][0]:(C[x][0] + 10), C[x][1]:(C[x][1] + 10)][:,:,0])
    return list_red_regions

In [5]:
from scipy import sparse
def test_lower_star_img(img):
    """
    Construct a lower star filtration on an image

    Parameters
    ----------
    img: ndarray (M, N)
        An array of single channel image data

    Returns
    -------
    I: ndarray (K, 2)
        A 0-dimensional persistence diagram corresponding to the sublevelset filtration
    """
    m, n = img.shape

    idxs = np.arange(m * n).reshape((m, n))

    I = idxs.flatten()
    J = idxs.flatten()
    V = img.flatten()

    # Connect 8 spatial neighbors
    tidxs = np.ones((m + 2, n + 2), dtype=np.int64) * np.nan
    tidxs[1:-1, 1:-1] = idxs

    tD = np.ones_like(tidxs) * np.nan
    tD[1:-1, 1:-1] = img

    for di in [-1, 0, 1]:
        for dj in [-1, 0, 1]:

            if di == 0 and dj == 0:
                continue

            thisJ = np.roll(np.roll(tidxs, di, axis=0), dj, axis=1)
            thisD = np.roll(np.roll(tD, di, axis=0), dj, axis=1)
            thisD = np.maximum(thisD, tD)

            # Deal with boundaries
            boundary = ~np.isnan(thisD)
            thisI = tidxs[boundary]
            thisJ = thisJ[boundary]
            thisD = thisD[boundary]

            I = np.concatenate((I, thisI.flatten()))
            J = np.concatenate((J, thisJ.flatten()))
            V = np.concatenate((V, thisD.flatten()))

    sparseDM = sparse.coo_matrix((V, (I, J)), shape=(idxs.size, idxs.size))

    return ripser(sparseDM, distance_matrix=True, maxdim=1)["dgms"]

In [3]:
from PIL import Image, ImageOps
from numpy import asarray
img = Image.open(r"C:\Users\hafid\OneDrive\Documents\Classes\MATH4020 Project\PNGs\TDA from previous project\train\frisen_five\06306895_20181106_Color_0 OD grade V_result.png")
img = img.resize((224, 224))
img = ImageOps.grayscale(img)
# plt.imshow(img)
img.show()

# keep track of image #


# img_array = asarray(img)
# plt.imshow(img_array)
# plt.gray()
# plt.imshow(1/255 * img_array[:,:,2])

# How to store/open all images (crop?)
# Find range of all possible image sizes

# open all images and resize them to (224, 224) all in grey scale
# make 3d np array with length (224, 224, n) -- n = number of images

In [34]:
import os
from os import listdir
import pickle
import matplotlib.pyplot as plt

data_folder_path = r"C:\Users\hafid\OneDrive\Documents\Classes\MATH4020 Project\PNGs\TDA from previous project\train"

def list_of_folders(folder_path):
    """ Get list of subfolders from main folder path
        Args: path of main folder (folder_path -- string)
        Returns: list of subfolders (folder_dir_list)
    """
    folder_dir_list = os.listdir(folder_path)
    return folder_dir_list

folders = list_of_folders(data_folder_path)
folders

['frisen_five', 'frisen_four', 'frisen_one', 'frisen_three', 'frisen_two']

In [1]:
import os
import numpy as np
from PIL import Image, ImageOps


def process_images(path, new_size):
    """ Process all images (png's) found in given path by resizing each image
        and converting to grayscale
    
    """
    images = []
    for filename in os.listdir(path):
        if filename.endswith(".png"):
            img_path = os.path.join(path, filename)
            img = Image.open(img_path)
            img_resized = img.resize((new_size, new_size))
            
            # Convert to grayscale
            img_gray = ImageOps.grayscale(img_resized)
            images.append(np.array(img_gray))
    return np.stack(images, axis=-1)  # Stack images along the last axis

In [3]:
dire = r"C:\Users\hafid\OneDrive\Documents\Classes\MATH4020 Project\PNGs\TDA from previous project\before_split\frisen_five_with_rotations"
rot_fives = process_images(dire, 224)

In [6]:
import pickle

with open('rot_fives.pkl', 'wb') as f:
    pickle.dump(rot_fives, f)

In [1]:
# Root folder containing subfolders
root_folder = r"C:\Users\hafid\OneDrive\Documents\Classes\MATH4020 Project\PNGs\TDA from previous project\train"

# Process each subfolder
all_arrays = []
for subfolder in folders:
    subfolder_path = os.path.join(root_folder, subfolder)
    images_array = process_images(subfolder_path)
    all_arrays.append(images_array)

# Print the shape of each array
for i, images_array in enumerate(all_arrays):
    print(f'Shape of array for type{i+1}: {images_array.shape}')

NameError: name 'folders' is not defined

In [12]:
for i, images_array in enumerate(all_arrays):
    print(f'Shape of array for type{i+1}: {images_array.shape}')

Shape of array for type1: (224, 224, 62)


In [31]:
zeros_folder = r"C:\Users\hafid\OneDrive\Documents\Classes\MATH4020 Project\PNGs\TDA from previous project\IIHTT Frisen Zeroes"

frisen_zero_array = []
images_array = process_images(zeros_folder)
frisen_zero_array.append(images_array)

print(f'Shape of array for frisen zero: {frisen_zero_array.shape}')

In [10]:
for i, images_array in enumerate(frisen_zero_array):
    print(f'Shape of frisen zero array: {images_array.shape}')

Shape of frisen zero array: (224, 224, 659)


In [50]:
folders[4]

'frisen_two'

In [36]:
data_folder_path = r"C:\Users\hafid\OneDrive\Documents\Classes\MATH4020 Project\PNGs\TDA from previous project\before_split"

In [51]:
subfolder_path = os.path.join(data_folder_path, folders[4])
images_array = process_images(subfolder_path, 224)

In [52]:
# Save the arrays using pickle
with open('frisen_twos_all.pkl', 'wb') as f:
    pickle.dump(images_array, f)

In [66]:
# To load the arrays later:
with open('processed_images.pkl', 'rb') as f:
    loaded_arrays = pickle.load(f)

test_img = loaded_arrays[0][:,:,0]
# print(test_img)
gray_img = Image.fromarray(test_img)
gray_img.show()

In [63]:
test_img = all_arrays[0][:,:,0]
print(test_img)
# plt.imshow(test_img)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [64]:
g_img = Image.fromarray(test_img)
g_img.show()